# Reinforcement

R-5.1 Execute the experiment from Code Fragment 5.1 and compare the results on your system to those we report in Code Fragment 5.2.

In [2]:
import sys

def test_list_memory_usage(n):
    data=[]
    for k in range(n):
        a = len(data)
        b = sys.getsizeof(data)
        print("Length: {0:3d}; Size in bytes: {1:4d}".format(a, b))
        data.append(None)

test_list_memory_usage(26)

# Initial size of empty list is 56 bytes (more efficient than that of the book)
# First added element increases list size by: 32 bytes (same as book)
# 5th element increases list size by: 32 bytes (showing that architecture of system is 64-bit 
# because 4 addresses stored in the list equals 32 bytes (8 bytes * 4 addresses))
# 

Length:   0; Size in bytes:   56
Length:   1; Size in bytes:   88
Length:   2; Size in bytes:   88
Length:   3; Size in bytes:   88
Length:   4; Size in bytes:   88
Length:   5; Size in bytes:  120
Length:   6; Size in bytes:  120
Length:   7; Size in bytes:  120
Length:   8; Size in bytes:  120
Length:   9; Size in bytes:  184
Length:  10; Size in bytes:  184
Length:  11; Size in bytes:  184
Length:  12; Size in bytes:  184
Length:  13; Size in bytes:  184
Length:  14; Size in bytes:  184
Length:  15; Size in bytes:  184
Length:  16; Size in bytes:  184
Length:  17; Size in bytes:  248
Length:  18; Size in bytes:  248
Length:  19; Size in bytes:  248
Length:  20; Size in bytes:  248
Length:  21; Size in bytes:  248
Length:  22; Size in bytes:  248
Length:  23; Size in bytes:  248
Length:  24; Size in bytes:  248
Length:  25; Size in bytes:  312


R-5.2 In Code Fragment 5.1,we perform an experiment to compare the length of a Python list to its underlying memory usage. Determining the sequence of array sizes requires a manual inspection of the output of that program. Redesign the experiment so that the program outputs only those values of k at which the existing capacity is exhausted. For example, on a system consistent with the results of Code Fragment 5.2, your program should output that the sequence of array capacities are 0, 4, 8, 16, 25, . . . .

In [9]:
import sys

def list_memory_usage(n):
    data=[]
    previous_size = None
    previous_length = None
    for _ in range(n):
        a = len(data)
        b = sys.getsizeof(data)

        if not previous_size:
            previous_size = b
    
        if b > previous_size:
            print("Length: {0:3d}; Size in bytes: {1:4d}; Diff: {2:4d}".format(previous_length, previous_size, b-previous_size))
            previous_size = b

        data.append(None)
        previous_length = a

list_memory_usage(1000)

Length:   0; Size in bytes:   56; Diff:   32
Length:   4; Size in bytes:   88; Diff:   32
Length:   8; Size in bytes:  120; Diff:   64
Length:  16; Size in bytes:  184; Diff:   64
Length:  24; Size in bytes:  248; Diff:   64
Length:  32; Size in bytes:  312; Diff:   64
Length:  40; Size in bytes:  376; Diff:   96
Length:  52; Size in bytes:  472; Diff:   96
Length:  64; Size in bytes:  568; Diff:   96
Length:  76; Size in bytes:  664; Diff:  128
Length:  92; Size in bytes:  792; Diff:  128
Length: 108; Size in bytes:  920; Diff:  160
Length: 128; Size in bytes: 1080; Diff:  160
Length: 148; Size in bytes: 1240; Diff:  192
Length: 172; Size in bytes: 1432; Diff:  224
Length: 200; Size in bytes: 1656; Diff:  256
Length: 232; Size in bytes: 1912; Diff:  288
Length: 268; Size in bytes: 2200; Diff:  320
Length: 308; Size in bytes: 2520; Diff:  352
Length: 352; Size in bytes: 2872; Diff:  384
Length: 400; Size in bytes: 3256; Diff:  448
Length: 456; Size in bytes: 3704; Diff:  512
Length: 52

R-5.3 Modify the experiment from Code Fragment 5.1 in order to demonstrate that Python’s list class occasionally shrinks the size of its underlying array when elements are popped from a list.

In [11]:
import sys

def test_list_shrinking_memory_usage(n):
    data=[]
    for k in range(n):
        a = len(data)
        b = sys.getsizeof(data)
        print("Length: {0:3d}; Size in bytes: {1:4d}".format(a, b))
        data.append(None)
    print("\nTesting shrinking:\n\n")
    for k in range((n//4)*3):
        data.pop()
        a = len(data)
        b = sys.getsizeof(data)
        print("Length: {0:3d}; Size in bytes: {1:4d}".format(a, b))

test_list_shrinking_memory_usage(50)

# Between length 25 and length 49, Python increased the list size twice,
# however, Python only shrink the size of the list 1 time when going from 49 to 25
# Note that when shrinking, overall size on length 15 is 216 compared to 184 and 248 on length 17,
# so it seems like it does some mathematical solution to define the size.

Length:   0; Size in bytes:   56
Length:   1; Size in bytes:   88
Length:   2; Size in bytes:   88
Length:   3; Size in bytes:   88
Length:   4; Size in bytes:   88
Length:   5; Size in bytes:  120
Length:   6; Size in bytes:  120
Length:   7; Size in bytes:  120
Length:   8; Size in bytes:  120
Length:   9; Size in bytes:  184
Length:  10; Size in bytes:  184
Length:  11; Size in bytes:  184
Length:  12; Size in bytes:  184
Length:  13; Size in bytes:  184
Length:  14; Size in bytes:  184
Length:  15; Size in bytes:  184
Length:  16; Size in bytes:  184
Length:  17; Size in bytes:  248
Length:  18; Size in bytes:  248
Length:  19; Size in bytes:  248
Length:  20; Size in bytes:  248
Length:  21; Size in bytes:  248
Length:  22; Size in bytes:  248
Length:  23; Size in bytes:  248
Length:  24; Size in bytes:  248
Length:  25; Size in bytes:  312
Length:  26; Size in bytes:  312
Length:  27; Size in bytes:  312
Length:  28; Size in bytes:  312
Length:  29; Size in bytes:  312
Length:  3

R-5.4 Our Dynamic Array class, as given in CodeFragment 5.3, does not support use of negative indices with `__getitem__`. Update that method to better match the semantics of a Python list.

In [19]:
import ctypes # provides low-level arrays 

class DynamicArray:

    def __init__(self):
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        if k < 0:
            k = self._n + k
    
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        return self._A[k]

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, c):
        B = self._make_array(c)
        for k in range(self._n):
            B[k] = self._A[k]
        self._A = B
        self._capacity = c

    def _make_array(self, c):
        return (c * ctypes.py_object)()

a = DynamicArray()
for i in range(10):
    a.append(i)
    print(i, end=" ")
print()
print(a[-1])
print(a[-3])
print(a[-len(a)])
print(a[0])
print(a[1])

0 1 2 3 4 5 6 7 8 9 
9
7
0
0
1


R-5.5 Redo the justification of Proposition 5.1 assuming that the the cost of growing the array from size k to size 2k is 3k cyber-dollars. How much should each append operation be charged to make the amortization work?

Each append operation should be charged: 7.

Assuming that 1 cyber-dollar is enough to pay for the execution of each append operation, and assuming that the cost of growing the array from size k to size 2k is 3k cyber dollars, we can determine how many cyber dollars we need to charge by use of the following example:

If we have a list of 8 elements, where the cost of going from 4 to 8 elements has been paid, and we need to have saved enough in the 4 elements that have been added (from the 5th to the 8th element), and we also know that we need to duplicate the size of the list, being k=8, it is going to cost 3\*8 = 24 cyber-dollars, then we need to have saved 24/4 = 6 cyber-dollars per element. 

So, if we charge 7 cyber-dollars per append operation, in the example where k=8, we would have 7\*4 - 6\*4 = 24, where 12\*4 is the total amount of cyber-dollars charged from the 5th to the 8th element, and 1*4 is the amount of cyber-dollars used already to append the 5th to 8th elements. Thus, leaving with a credit of 24 cyber-dollars to be used when the array needs to grow from k to 2k, that is from 8 to 16 elements in this example.

R-5.6 Our implementation of insert for the DynamicArray class, as given in Code Fragment 5.5, has the following inefficiency. In the case when a resize occurs, the resize operation takes time to copy all the elements from an old array to a new array, and then the subsequent loop in the body of insert shifts many of those elements. Give an improved implementation of the insert method, so that, in the case of a resize, the elements are shifted into their final position during that operation, thereby avoiding the subsequent shifting.

In [36]:
import ctypes

class DynamicArray:

    def __init__(self):
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        if k < 0:
            k = self._n + k
    
        if not 0 <= k < self._n:
            raise IndexError("invalid index")
        return self._A[k]

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def _resize(self, c, k = -1):
        B = self._make_array(c)
        shift = 1 if k > -1 else 0
    
        for i in range(k):
            B[i] = self._A[i]
    
        for j in range(self._n-1, k-1, -1):
            B[j+shift] = self._A[j]
    
        self._A = B
        self._capacity = c

    def _make_array(self, c):
        return (c * ctypes.py_object)()
        #return [None] * c  # for debugging
    
    def insert(self, k, value):
        if k < 0:
            k = self._n + 1 + k

        if self._n == self._capacity:
            self._resize(2 * self._capacity, k)
        else:
            for j in range(self._n, k, -1):
                self._A[j] = self._A[j-1]

        self._A[k] = value
        self._n += 1

    def __str__(self):
        return ",".join(str(self._A[x]) for x in range(self._n))


a = DynamicArray()
for i in range(5):  
    a.append(i)
print(a)
a.insert(0,5)
a.insert(3,6)
a.insert(7,7)
a.insert(4,8)
a.insert(-2,9)
print(a, a._capacity)



0,1,2,3,4
5,0,1,6,8,2,3,4,9,7 16
